In [21]:
%matplotlib inline

import QSTK.qstkutil.qsdateutil as du
import QSTK.qstkutil.tsutil as tsu
import QSTK.qstkutil.DataAccess as da

import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from cvxopt import matrix
import cvxopt

In [2]:
def simulate(startdate, enddate, ls_symbols, ls_allocations):
    dt_start = startdate
    dt_end = enddate
    dt_timeofday = dt.timedelta(hours=16)
    ldt_timestamps = du.getNYSEdays(dt_start, dt_end, dt_timeofday)
    c_dataobj = da.DataAccess("Yahoo", cachestalltime=0)
    ls_keys = ["close"]
    ldf_data = c_dataobj.get_data(ldt_timestamps, ls_symbols, ls_keys)
    d_data = dict(zip(ls_keys, ldf_data))
    stock_list = []
    for i,j in zip(ls_symbols,ls_allocations):
        price = d_data["close"][str(i)].values
        cumu_ret = price / price[0]
        invest = cumu_ret*float(j)
        stock_list.append(invest)
    Fund_invest = sum(stock_list)
    Fund_cumu_ret = Fund_invest/Fund_invest[0]
    Fund_daily_return = [0]
    for i in range(1,len(Fund_cumu_ret)):
        rt = (Fund_cumu_ret[i]/Fund_cumu_ret[i-1])-1
        Fund_daily_return.append(rt)
    
    Annual_ret =  Fund_invest[-1]/ Fund_invest[0]
    M_daily_ret = np.mean(Fund_daily_return)
    SD_dail_ret = np.std(Fund_daily_return)
    Sharpe_R = np.sqrt(252)*(M_daily_ret/SD_dail_ret)
    
    return Annual_ret, M_daily_ret, SD_dail_ret, Sharpe_R

In [4]:
ls_symbols = ['AXP', 'HPQ', 'IBM', 'HNZ']
allocation = [0.0, 0.0, 0.0, 1.0]
startdate = dt.datetime(2010,1,1)
enddate = dt.datetime(2010,12, 31)

a,b,c,d = simulate(startdate, enddate, ls_symbols, allocation)
print(a,b,c,d)

(1.1960583567954952, 0.0007562855855930262, 0.009242992559366399, 1.2988933400758884)


In [22]:
def ret_optimal_allocation(startdate, enddate, ls_symbols):
    dt_start = startdate
    dt_end = enddate
    dt_timeofday = dt.timedelta(hours=16)
    ldt_timestamps = du.getNYSEdays(dt_start, dt_end, dt_timeofday)
    c_dataobj = da.DataAccess("Yahoo", cachestalltime=0)
    ls_keys = ["close"]
    ldf_data = c_dataobj.get_data(ldt_timestamps, ls_symbols, ls_keys)
    d_data = dict(zip(ls_keys, ldf_data))
    beta_list = []
    for i in ls_symbols:
        price = d_data["close"][str(i)].values
        ann_cumu_ret = price / price[0]
        beta_list.append(ann_cumu_ret)
    # minimum expected return threshold
    r_min = 0.10
    r_avg = matrix(np.array(beta_list))
    # inequality constraints Gx <= h
    # captures the constraints (avg_ret'x >= r_min) and (x >= 0)
    n = len(ls_symbols)
    G = matrix(np.concatenate((np.transpose(np.array(r_avg)), np.identity(n)), 0))
    h = matrix(np.concatenate((np.ones((1,1))*r_min, np.zeros((n,1))), 0))
    # equality constraint Ax = b; captures the constraint sum(x) == 1
    A = matrix(1.0, (1,n))
    b = matrix(1.0)
    sol = solvers.qp(G, h, A, b)
    print sol

In [23]:
ret_opt = ret_optimal_allocation(startdate, enddate, ls_symbols)

NameError: global name 'solvers' is not defined